In [3]:
from random import seed, randint
from numpy import array, argmax
from math import ceil, log10, sqrt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, TimeDistributed, RepeatVector

In [4]:
def random_sum_pairs(n_examples, n_numbers, largest):
    X, y = [], []
    for i in range(n_examples):
        in_pattern = [randint(1, largest) for _ in range(n_numbers)]
        out_pattern = sum(in_pattern)
        X.append(in_pattern)
        y.append(out_pattern)
    return X, y

In [5]:
X, y = random_sum_pairs(100, 4, 20)
print(X)

[[9, 19, 8, 8], [1, 2, 6, 5], [7, 8, 20, 7], [20, 15, 14, 9], [3, 19, 9, 7], [15, 14, 12, 15], [1, 9, 17, 16], [5, 14, 7, 4], [14, 10, 11, 16], [20, 12, 17, 16], [5, 6, 10, 17], [10, 10, 3, 16], [12, 9, 4, 15], [12, 9, 7, 14], [9, 10, 1, 19], [17, 7, 12, 4], [4, 1, 11, 4], [14, 1, 15, 14], [5, 20, 15, 9], [18, 6, 5, 11], [3, 10, 7, 1], [14, 5, 1, 10], [11, 8, 19, 13], [15, 10, 6, 13], [12, 10, 5, 8], [13, 20, 13, 4], [19, 15, 2, 11], [5, 17, 6, 10], [18, 2, 15, 4], [10, 14, 3, 20], [5, 10, 9, 1], [6, 8, 16, 5], [6, 19, 19, 18], [5, 14, 15, 20], [2, 12, 12, 17], [8, 17, 15, 15], [16, 12, 15, 11], [13, 11, 6, 1], [20, 18, 12, 13], [12, 11, 14, 1], [5, 5, 9, 20], [4, 3, 1, 4], [6, 11, 13, 7], [18, 3, 20, 2], [18, 17, 17, 4], [13, 9, 14, 7], [20, 17, 11, 2], [11, 14, 4, 17], [16, 4, 15, 15], [12, 13, 17, 14], [18, 20, 16, 19], [1, 19, 12, 17], [8, 12, 18, 18], [6, 18, 10, 7], [15, 1, 19, 17], [6, 19, 11, 7], [14, 5, 2, 12], [12, 7, 9, 13], [2, 20, 3, 4], [18, 4, 20, 5], [17, 12, 8, 19], [1

In [6]:
print(y)

[44, 14, 42, 58, 38, 56, 43, 30, 51, 65, 38, 39, 40, 42, 39, 40, 20, 44, 49, 40, 21, 30, 51, 44, 35, 50, 47, 38, 39, 47, 25, 35, 62, 54, 43, 55, 54, 31, 63, 38, 39, 12, 37, 43, 56, 43, 50, 46, 50, 56, 73, 49, 56, 41, 52, 43, 33, 41, 29, 47, 56, 53, 39, 50, 47, 59, 59, 32, 47, 27, 51, 39, 57, 51, 65, 53, 43, 44, 19, 60, 53, 36, 49, 26, 34, 61, 49, 37, 47, 23, 40, 46, 46, 60, 48, 53, 33, 36, 56, 29]


In [7]:
ceil(log10(20+1))

2

In [17]:
def pairs_to_string(X, y, n_numbers, largest):
    max_length = n_numbers * ceil(log10(largest+1)) + n_numbers - 1
    Xstr = []
    for p in X:
        strp = '+'.join([str(n) for n in p])
        strp = ''.join([' ' for _ in range(max_length - len(strp))]) + strp
        Xstr.append(strp)
    max_length = ceil(log10(n_numbers * (largest+1)))
    ystr = []
    for p in y:
        strp = str(p)
        strp = ''.join([' ' for _ in range(max_length - len(strp))]) + strp
        ystr.append(strp)
    return Xstr, ystr

In [18]:
pairs_to_string(X, y, 4, 20)

(['   9+19+8+8',
  '    1+2+6+5',
  '   7+8+20+7',
  ' 20+15+14+9',
  '   3+19+9+7',
  '15+14+12+15',
  '  1+9+17+16',
  '   5+14+7+4',
  '14+10+11+16',
  '20+12+17+16',
  '  5+6+10+17',
  ' 10+10+3+16',
  '  12+9+4+15',
  '  12+9+7+14',
  '  9+10+1+19',
  '  17+7+12+4',
  '   4+1+11+4',
  ' 14+1+15+14',
  '  5+20+15+9',
  '  18+6+5+11',
  '   3+10+7+1',
  '  14+5+1+10',
  ' 11+8+19+13',
  ' 15+10+6+13',
  '  12+10+5+8',
  ' 13+20+13+4',
  ' 19+15+2+11',
  '  5+17+6+10',
  '  18+2+15+4',
  ' 10+14+3+20',
  '   5+10+9+1',
  '   6+8+16+5',
  ' 6+19+19+18',
  ' 5+14+15+20',
  ' 2+12+12+17',
  ' 8+17+15+15',
  '16+12+15+11',
  '  13+11+6+1',
  '20+18+12+13',
  ' 12+11+14+1',
  '   5+5+9+20',
  '    4+3+1+4',
  '  6+11+13+7',
  '  18+3+20+2',
  ' 18+17+17+4',
  '  13+9+14+7',
  ' 20+17+11+2',
  ' 11+14+4+17',
  ' 16+4+15+15',
  '12+13+17+14',
  '18+20+16+19',
  ' 1+19+12+17',
  ' 8+12+18+18',
  '  6+18+10+7',
  ' 15+1+19+17',
  '  6+19+11+7',
  '  14+5+2+12',
  '  12+7+9+13',
  '   2+20+3+4

In [24]:
Xstr, ystr = pairs_to_string(X, y, 4, 20)

In [25]:
def integer_encode(X, y, alphabets):
    char_to_int = dict((c, i) for i, c in enumerate(alphabets))
    print(char_to_int)
    Xenc = []
    for p in X:
        integer_encoded = [char_to_int[char] for char in p]
        Xenc.append(integer_encoded)
    yenc = []
    for p in y:
        integer_encoded = [char_to_int[char] for char in p]
        yenc.append(integer_encoded)
    return Xenc, yenc

In [26]:
integer_encode(Xstr, ystr, ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0', ' ', '+'])

{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '0': 9, ' ': 10, '+': 11}


([[10, 10, 10, 8, 11, 0, 8, 11, 7, 11, 7],
  [10, 10, 10, 10, 0, 11, 1, 11, 5, 11, 4],
  [10, 10, 10, 6, 11, 7, 11, 1, 9, 11, 6],
  [10, 1, 9, 11, 0, 4, 11, 0, 3, 11, 8],
  [10, 10, 10, 2, 11, 0, 8, 11, 8, 11, 6],
  [0, 4, 11, 0, 3, 11, 0, 1, 11, 0, 4],
  [10, 10, 0, 11, 8, 11, 0, 6, 11, 0, 5],
  [10, 10, 10, 4, 11, 0, 3, 11, 6, 11, 3],
  [0, 3, 11, 0, 9, 11, 0, 0, 11, 0, 5],
  [1, 9, 11, 0, 1, 11, 0, 6, 11, 0, 5],
  [10, 10, 4, 11, 5, 11, 0, 9, 11, 0, 6],
  [10, 0, 9, 11, 0, 9, 11, 2, 11, 0, 5],
  [10, 10, 0, 1, 11, 8, 11, 3, 11, 0, 4],
  [10, 10, 0, 1, 11, 8, 11, 6, 11, 0, 3],
  [10, 10, 8, 11, 0, 9, 11, 0, 11, 0, 8],
  [10, 10, 0, 6, 11, 6, 11, 0, 1, 11, 3],
  [10, 10, 10, 3, 11, 0, 11, 0, 0, 11, 3],
  [10, 0, 3, 11, 0, 11, 0, 4, 11, 0, 3],
  [10, 10, 4, 11, 1, 9, 11, 0, 4, 11, 8],
  [10, 10, 0, 7, 11, 5, 11, 4, 11, 0, 0],
  [10, 10, 10, 2, 11, 0, 9, 11, 6, 11, 0],
  [10, 10, 0, 3, 11, 4, 11, 0, 11, 0, 9],
  [10, 0, 0, 11, 7, 11, 0, 8, 11, 0, 2],
  [10, 0, 4, 11, 0, 9, 11, 5, 11, 0,

In [27]:
def one_hot_encode(X, y, max_int):
    Xenc = []
    for p in X:
        pattern = []
        for index in p:
            vector = [0 for _ in range(max_int)]
            vector[index] = 1
            pattern.append(vector)
        Xenc.append(pattern)
    yenc = []
    for p in y:
        pattern = []
        for index in p:
            vector = [0 for _ in range(max_int)]
            vector[index] = 1
            pattern.append(vector)
        yenc.append(pattern)
    return Xenc, yenc

In [34]:
def generate_data(n_samples, n_numbers, largest, alphabets):
    X, y = random_sum_pairs(n_samples, n_numbers, largest)
    X, y = pairs_to_string(X, y, n_numbers, largest)
    X, y = integer_encode(X, y, alphabets)
    X, y = one_hot_encode(X, y, len(alphabets))
    X, y = array(X), array(y)
    return X, y

In [29]:
def integer_decode(seq, alphabets):
    int_to_char = dict((i, c) for i, c in enumerate(alphabets))
    strings = []
    for p in seq:
        string = int_to_char[argmax(p)]
        strings.append(string)
    return ''.join(strings)

In [30]:
seed(1)
n_samples = 1000
n_numbers = 3
largest = 20
alphabets = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0', ' ', '+']
n_chars = len(alphabets)
n_in_seq_length = n_numbers * ceil(log10(largest+1)) + n_numbers - 1
n_out_seq_length = ceil(log10(n_numbers * (largest+1)))

In [39]:
n_chars

12

In [31]:
n_in_seq_length

8

In [32]:
n_out_seq_length

2

In [35]:
X, y = generate_data(n_samples, n_numbers, largest, alphabets)

{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '0': 9, ' ': 10, '+': 11}


In [37]:
X.shape

(1000, 8, 12)

In [38]:
X[0]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]])

In [41]:
model = Sequential()
model.add(LSTM(128, input_shape=(n_in_seq_length, n_chars)))
model.add(RepeatVector(n_out_seq_length))
model.add(LSTM(64, return_sequences=True))
model.add(TimeDistributed(Dense(n_chars, activation='softmax')))

In [42]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [43]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 2, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 2, 64)             49408     
_________________________________________________________________
time_distributed (TimeDistri (None, 2, 12)             780       
Total params: 122,380
Trainable params: 122,380
Non-trainable params: 0
_________________________________________________________________


In [45]:
for i in range(200):
    X, y = generate_data(n_samples, n_numbers, largest, alphabets)
    print('Epoch: ', i)
    model.fit(X, y, epochs=5, batch_size=50)

{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '0': 9, ' ': 10, '+': 11}
Epoch:  0
20/20 [==============================] - 0s 9ms/step - loss: 1.4429 - accuracy: 0.4365
{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '0': 9, ' ': 10, '+': 11}
Epoch:  1
20/20 [==============================] - 0s 9ms/step - loss: 1.4065 - accuracy: 0.4540
{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '0': 9, ' ': 10, '+': 11}
Epoch:  2
20/20 [==============================] - 0s 9ms/step - loss: 1.4127 - accuracy: 0.4600
{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '0': 9, ' ': 10, '+': 11}
Epoch:  3
20/20 [==============================] - 0s 10ms/step - loss: 1.4199 - accuracy: 0.4355
{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '0': 9, ' ': 10, '+': 11}
Epoch:  4
20/20 [==============================] - 0s 10ms/step - loss: 1.3964 - accuracy: 0.4660
{'1': 0, '2': 1, '

20/20 [==============================] - 0s 8ms/step - loss: 1.0785 - accuracy: 0.6260
{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '0': 9, ' ': 10, '+': 11}
Epoch:  42
20/20 [==============================] - 0s 9ms/step - loss: 1.0512 - accuracy: 0.6115
{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '0': 9, ' ': 10, '+': 11}
Epoch:  43
20/20 [==============================] - 0s 8ms/step - loss: 1.0437 - accuracy: 0.6260
{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '0': 9, ' ': 10, '+': 11}
Epoch:  44
20/20 [==============================] - 0s 9ms/step - loss: 0.9863 - accuracy: 0.6710
{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '0': 9, ' ': 10, '+': 11}
Epoch:  45
20/20 [==============================] - 0s 9ms/step - loss: 1.0005 - accuracy: 0.6535
{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '0': 9, ' ': 10, '+': 11}
Epoch:  46
20/20 [========

Epoch:  83
20/20 [==============================] - 0s 11ms/step - loss: 0.5892 - accuracy: 0.8100
{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '0': 9, ' ': 10, '+': 11}
Epoch:  84
20/20 [==============================] - 0s 9ms/step - loss: 0.5745 - accuracy: 0.8240
{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '0': 9, ' ': 10, '+': 11}
Epoch:  85
20/20 [==============================] - 0s 8ms/step - loss: 0.5711 - accuracy: 0.8350
{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '0': 9, ' ': 10, '+': 11}
Epoch:  86
20/20 [==============================] - 0s 9ms/step - loss: 0.5306 - accuracy: 0.8795
{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '0': 9, ' ': 10, '+': 11}
Epoch:  87
20/20 [==============================] - 0s 14ms/step - loss: 0.5807 - accuracy: 0.8040
{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '0': 9, ' ': 10, '+': 11}
Epoch:  88
20

20/20 [==============================] - 0s 11ms/step - loss: 0.3285 - accuracy: 0.9605
{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '0': 9, ' ': 10, '+': 11}
Epoch:  125
20/20 [==============================] - 0s 8ms/step - loss: 0.3507 - accuracy: 0.9365
{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '0': 9, ' ': 10, '+': 11}
Epoch:  126
20/20 [==============================] - 0s 8ms/step - loss: 0.4400 - accuracy: 0.8360
{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '0': 9, ' ': 10, '+': 11}
Epoch:  127
20/20 [==============================] - 0s 8ms/step - loss: 0.3858 - accuracy: 0.8930
{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '0': 9, ' ': 10, '+': 11}
Epoch:  128
20/20 [==============================] - 0s 8ms/step - loss: 0.3339 - accuracy: 0.9510
{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '0': 9, ' ': 10, '+': 11}
Epoch:  129
20/20 [==

20/20 [==============================] - 0s 9ms/step - loss: 0.2046 - accuracy: 0.9865
{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '0': 9, ' ': 10, '+': 11}
Epoch:  166
20/20 [==============================] - 0s 9ms/step - loss: 0.2001 - accuracy: 0.9810
{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '0': 9, ' ': 10, '+': 11}
Epoch:  167
20/20 [==============================] - 0s 10ms/step - loss: 0.2121 - accuracy: 0.9695
{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '0': 9, ' ': 10, '+': 11}
Epoch:  168
20/20 [==============================] - 0s 8ms/step - loss: 0.2311 - accuracy: 0.9580
{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '0': 9, ' ': 10, '+': 11}
Epoch:  169
20/20 [==============================] - 0s 8ms/step - loss: 0.3515 - accuracy: 0.8510
{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '0': 9, ' ': 10, '+': 11}
Epoch:  170
20/20 [==

In [46]:
X, y = generate_data(n_samples, n_numbers, largest, alphabets)
result = model.predict(X, batch_size=50)

{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '0': 9, ' ': 10, '+': 11}


In [54]:
result[0]

array([[2.7544391e-07, 3.6060780e-02, 9.6393782e-01, 1.1435411e-06,
        5.7647895e-09, 1.6573215e-11, 9.8591036e-13, 1.0626995e-13,
        4.6423260e-10, 9.6303665e-10, 3.5030381e-12, 1.0559099e-11],
       [6.3744009e-02, 4.2560959e-04, 4.1782856e-05, 9.4592706e-06,
        2.0401542e-06, 7.3339336e-08, 1.6963864e-07, 1.3375979e-05,
        2.3503745e-02, 9.1225958e-01, 8.7947633e-08, 1.4953256e-08]],
      dtype=float32)

In [55]:
integer_decode(result[0], alphabets)

'30'

In [56]:
expected = [integer_decode(y1, alphabets) for y1 in y]
predicted = [integer_decode(y1, alphabets) for y1 in result]

In [60]:
list(zip(expected, predicted))

[('30', '30'),
 ('54', '53'),
 ('33', '33'),
 ('41', '41'),
 ('22', '22'),
 ('23', '23'),
 ('33', '33'),
 ('35', '35'),
 ('26', '26'),
 ('23', '23'),
 ('43', '43'),
 ('23', '23'),
 ('32', '32'),
 ('54', '54'),
 ('50', '50'),
 ('18', '18'),
 ('51', '51'),
 ('26', '26'),
 ('15', '15'),
 ('30', '30'),
 ('27', '27'),
 ('12', '12'),
 ('42', '42'),
 ('31', '31'),
 ('30', '30'),
 ('43', '43'),
 ('15', '15'),
 ('33', '33'),
 ('19', '19'),
 ('32', '32'),
 ('50', '50'),
 ('52', '52'),
 ('24', '24'),
 ('44', '44'),
 ('47', '47'),
 ('34', '34'),
 ('42', '42'),
 ('23', '23'),
 ('23', '23'),
 ('32', '32'),
 ('37', '37'),
 ('26', '26'),
 ('26', '26'),
 ('35', '35'),
 ('33', '33'),
 ('13', '14'),
 ('21', '21'),
 ('31', '31'),
 ('20', '20'),
 ('39', '39'),
 ('28', '28'),
 ('32', '32'),
 ('28', '28'),
 ('14', '14'),
 ('25', '25'),
 ('30', '30'),
 ('23', '23'),
 ('41', '41'),
 ('32', '32'),
 ('32', '32'),
 ('37', '37'),
 ('18', '18'),
 ('39', '39'),
 ('36', '36'),
 ('22', '22'),
 ('35', '35'),
 ('11', '1

In [66]:
import numpy as np
np.abs(np.array(expected, dtype='int') - np.array(predicted, dtype='int')).sum() / len(expected)

0.025